# Airtable & GitHub Analysis

In [ ]:
import os
import json

from airtable import airtable
from github import Github

### Setup
This repository is using the GitHub API and accessing Airtable.
You will need to setup the following environment variables:
- AIRTABLE_API_KEY
- GITHUB_API_KEY
And configure tables properly under Airtable Setup.

In [ ]:
# Airtable Setup
airtable_api_key = os.environ.get('AIRTABLE_API_KEY')
ovio_table_base_id = 'appsmuzyRwrWD39Rj'
project_table = 'GithHub Repository'
causes_table = 'Causes'

at = airtable.Airtable(ovio_table_base_id, project_table, api_key=airtable_api_key)
airtable_projects = at.get_all()

airtable_causes = airtable.Airtable(ovio_table_base_id, causes_table, api_key=airtable_api_key)

In [ ]:
def get_cause(cause_id):
    return airtable_causes.get(cause_id)['fields']['Name']

In [ ]:
# GitHub interaction
g = Github(os.environ.get('GITHUB_API_KEY'))

In [ ]:
airtable_projects[0]

In [ ]:
## sample output
{
    "name": "google-type",
    "owner": "femmebot",
    "fullname": "femmebot~google-type",
    "description": "Collaborative typography project using select passages from Aesop's Fables set to Google Fonts",
    "avatar": "https://avatars0.githubusercontent.com/u/471514?v=4",
    "url": "https://github.com/femmebot/google-type",
    "pushed": "2017-12-30T19:17:37Z",
    "created": "2014-07-22T23:43:17Z",
    "size": 49214,
    "stars": 943,
    "forks": 108,
    "topics": [
        "css",
        "google-fonts",
        "responsive",
        "typography"
    ],
    "language": "CSS",
    "watchers": 50
}

In [ ]:
def format_github_repo(organization, repo):
    return {
        "name": repo.name,
        "owner": repo.owner.login,
        "organization": repo.organization.login,
        "organization_description": organization['fields']['Description'],
        "organization_causes": list(map(get_cause, organization['fields']['Causes'])),
        "fullname": repo.full_name.replace('/', '~'),
        "description": repo.description,
        "avatar": repo.organization.avatar_url,
        "url": repo.html_url,
        "pushed": repo.pushed_at.isoformat() + 'Z',
        "created": repo.created_at.isoformat() + 'Z',
        "size": repo.size,
        "stars": repo.stargazers_count,
        "forks": repo.forks_count,
        "topics": repo.topics or [],
        "language": repo.language or '',
        "watchers": repo.watchers_count
    }

In [ ]:
format_github_repo(airtable_projects[0], )

In [ ]:
formatted_repositories = []

In [ ]:
for organization in airtable_projects:
    try:
        print(organization['fields']['GitHub'])
        g_organization = g.get_organization(organization['fields']['GitHub'].replace('https://github.com/', ''))
    except:
        continue
    for repo in g_organization.get_repos():
        #TODO
        # if repo.not_active:
        #    pass
        try:
            formatted_repository = format_github_repo(organization, repo)
            formatted_repositories.append(formatted_repository)
        except:
            pass

In [ ]:
airtable_projects[0]

In [ ]:
len(formatted_repositories)

In [ ]:
with open('repo_test.json', 'w') as fd:
    fd.write(json.dumps(formatted_repositories))  